<a href="https://colab.research.google.com/github/fermuba/Procesamiento-de-Datos/blob/main/S6/Ejemplos_clase/Ejemplo_6_automatizando_peticiones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Ejemplo 6: Automatizando peticiones

### 1. Objetivos:
    - Usar todo lo que aprendimos para automatizar peticiones al API
    - Guardar nuestros resultados en un archivo tipo .csv

---
    
### 2. Desarrollo:

Veamos cómo usar todo lo que aprendimos para automatizar el proceso de realizar múltiples peticiones a la API, reunirlas en un `DataFrame` y guardarlo en un .csv:

In [1]:
# Cargamos librerias
import pandas as pd
import requests
import time

In [11]:
# Creamos nuestras variables y les colocamos la informacion para solicitar a la API
endpoint = 'https://api.nasa.gov/neo/rest/v1/neo/browse'
payload = {'api_key': 'sYrr0KtFovL8YGwcaljKqnYzCVJal8XZ3Q0rled6'}


In [12]:
# Creacion de diccionario para almacenar los datos de cada paquete solicitado
dict_datos = {}

# Solicitamos 10 pedidos de 10 paginas de la misma API
for i in range(0, 10):

    # Al ser un proceso automatico, nos aseguramos que no se corte el pedido y hay algun error
    try:
        time.sleep(5)   # pausa de 5seg para no generar problemas en servidor
        r = requests.get(endpoint, params=payload, timeout=5) # Le indicamos con timeout que espere 5seg

        if r.status_code == 200:
            json = r.json()

            # Guardamos los datos en el diccionario, cada pedido en una clave
            data = json['near_earth_objects']
            dict_datos[i] = data

            # Obtenemos la direccion para el endpoint de la siguiente pagina a descargar
            new_link = json['links']['next']
            endpoint = new_link
    except:
        continue

In [14]:
# Normalizacion de los datos
for key in dict_datos:
    normalized = pd.json_normalize(dict_datos[key])
    df = pd.DataFrame.from_dict(normalized)
    dict_datos[key] = df

In [15]:
# Creamos una lista de DataFrames para usarla en pd.concat
lista_de_dataframes = []

for key in dict_datos:
    lista_de_dataframes.append(dict_datos[key])

In [17]:
# Concatenamos los df de la lista
df_completo = pd.concat(lista_de_dataframes, axis=0).reset_index(drop=True)

In [18]:
df_completo

,id,neo_reference_id,name,name_limited,designation,nasa_jpl_url,absolute_magnitude_h,is_potentially_hazardous_asteroid,close_approach_data,is_sentry_object,...,orbital_data.perihelion_distance,orbital_data.perihelion_argument,orbital_data.aphelion_distance,orbital_data.perihelion_time,orbital_data.mean_anomaly,orbital_data.mean_motion,orbital_data.equinox,orbital_data.orbit_class.orbit_class_type,orbital_data.orbit_class.orbit_class_description,orbital_data.orbit_class.orbit_class_range
0,2000433,2000433,433 Eros (A898 PA),Eros,433,https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,10.41,False,"[{'close_approach_date': '1900-12-27', 'close_...",False,...,1.133430877277801,178.8953167089443,1.782924415693006,2460445.650244578800,334.7274812953938,.5597426756016515,J2000,AMO,Near-Earth asteroid orbits similar to that of ...,1.017 AU < q (perihelion) < 1.3 AU
1,2000719,2000719,719 Albert (A911 TB),Albert,719,https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,15.59,False,"[{'close_approach_date': '1909-08-21', 'close_...",False,...,1.194434733405151,156.2221413583312,4.077803107571985,2459955.944856209393,102.3722423709411,.2302801886354059,J2000,AMO,Near-Earth asteroid orbits similar to that of ...,1.017 AU < q (perihelion) < 1.3 AU
2,2000887,2000887,887 Alinda (A918 AA),Alinda,887,https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,13.88,False,"[{'close_approach_date': '1910-01-04', 'close_...",False,...,1.060514970150709,350.478081294877,3.883763462379203,2460678.627539692991,289.4757166026648,.2535681417064373,J2000,AMO,Near-Earth asteroid orbits similar to that of ...,1.017 AU < q (perihelion) < 1.3 AU
3,2001036,2001036,1036 Ganymed (A924 UB),Ganymed,1036,https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,9.25,False,"[{'close_approach_date': '1910-02-25', 'close_...",False,...,1.2451939773117,132.4833028454874,4.085791920309684,2460569.648416968831,321.6905687364655,.2264841253027731,J2000,AMO,Near-Earth asteroid orbits similar to that of ...,1.017 AU < q (perihelion) < 1.3 AU
4,2001221,2001221,1221 Amor (1932 EA1),Amor,1221,https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,17.38,False,"[{'close_approach_date': '1900-03-08', 'close_...",False,...,1.084248654678618,26.67573388569273,2.755201145123765,2460838.656012985569,197.6417532004091,.370548941445061,J2000,AMO,Near-Earth asteroid orbits similar to that of ...,1.017 AU < q (perihelion) < 1.3 AU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,2017511,2017511,17511 (1992 QN),NaN,17511,https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,17.32,False,"[{'close_approach_date': '1900-12-11', 'close_...",False,...,.7628962228780419,202.4259339028952,1.616522565793132,2460586.150141898483,218.9938603462328,.759526161476742,J2000,APO,Near-Earth asteroid orbits which cross the Ear...,a (semi-major axis) > 1.0 AU; q (perihelion) <...
196,2018106,2018106,18106 Blume (2000 NX3),Blume,18106,https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,17.78,False,"[{'close_approach_date': '1904-08-28', 'close_...",False,...,1.192326477990022,234.9856159308461,3.694483685416088,2460164.292422705901,60.95434796793649,.2580541601002199,J2000,AMO,Near-Earth asteroid orbits similar to that of ...,1.017 AU < q (perihelion) < 1.3 AU
197,2018109,2018109,18109 (2000 NG11),NaN,18109,https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,17.19,False,"[{'close_approach_date': '1902-10-30', 'close_...",False,...,1.188781876279454,319.2698012922049,2.572950217079807,2460293.101922902609,41.03588268357758,.3820914097592752,J2000,AMO,Near-Earth asteroid orbits similar to that of ...,1.017 AU < q (perihelion) < 1.3 AU
198,2018172,2018172,18172 (2000 QL7),NaN,18172,https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,15.69,False,"[{'close_approach_date': '1913-11-01', 'close_...",False,...,1.185496107980462,100.6887283879882,3.664554894079723,2460208.117912187188,50.21046490848196,.2609934504782839,J2000,AMO,Near-Earth asteroid orbits similar to that of ...,1.017 AU < q (perihelion) < 1.3

In [ ]:
# Guardamos nuestro df en un archivo CSV
df_completo.to_csv('../../Datasets/near_earth_objects-raw.csv') # Cambiar la ruta donde se quiera guardar

¡Listo! Ya tenemos nuestro dataset para procesar en la siguiente sesión. En este último reto vas a implementar tú mismo este proceso, con la única diferencia de que vas a pedir de la página 10 a la 110. ¡Mucha suerte!